# Exploratory Data Analysis

## Setup

In [1]:
import pandas as pd
import janitor
import os
from dotenv import load_dotenv
import time
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
from utils import get_table_from_supabase, add_gbp_columns, explode_lists

#get keys from env
load_dotenv()
url = os.getenv("SUPABASE_URL")
key = os.getenv("SUPABASE_KEY")

## Retrieving Data from Supabase and Building Dataframes

I will connect to Supabase and retrieve all records, in order to start building my analysis dataframes. I will create one dataframe for funder information, and another for grants and recipients information.

In [2]:
#get tables and build dataframes
tables = ["funders", "causes", "areas", "beneficiaries", "grants", "recipients",
               "funder_causes", "funder_areas", "funder_beneficiaries", "funder_grants", 
               "recipient_grants", "recipient_areas"]

for table in tables:
    if table == "recipients":
        globals()[table] = get_table_from_supabase(url, key, table, batch_size=50, filter_recipients=True)
    else:
        globals()[table] = get_table_from_supabase(url, key, table)

print(f"Loaded {len(funders)} funders, {len(recipients)} recipients, {len(grants)} grants")

Loaded 499 funders, 295 recipients, 12637 grants


In [3]:
funders_df = funders.copy()

#define table relationships for funders
funder_rels = [
    {
        "join_table": funder_causes,
        "lookup_table": causes,
        "key": "cause_id",
        "value_col": "cause_name",
        "result_col": "causes"
    },
    {
        "join_table": funder_areas,
        "lookup_table": areas,
        "key": "area_id",
        "value_col": "area_name",
        "result_col": "areas"
    },
    {
        "join_table": funder_beneficiaries,
        "lookup_table": beneficiaries,
        "key": "ben_id",
        "value_col": "ben_name",
        "result_col": "beneficiaries"
    }
]

#group and merge
for rel in funder_rels:
    grouped = rel["join_table"].merge(rel["lookup_table"], on=rel["key"])
    grouped = grouped.groupby("registered_num")[rel["value_col"]].apply(list).reset_index()
    grouped.columns = ["registered_num", rel["result_col"]]
    funders_df = funders_df.merge(grouped, on="registered_num", how="left")

#add grant statistics columns 
grants_stats = funder_grants.merge(grants, on="grant_id")
grants_agg = grants_stats.groupby("registered_num").agg({
    "grant_id": "count",
    "amount": ["sum", "mean", "median"]
}).reset_index()
grants_agg.columns = ["registered_num", "num_grants", "total_given", "avg_grant", "median_grant"]

funders_df = funders_df.merge(grants_agg, on="registered_num", how="left")
funders_df["num_grants"] = funders_df["num_grants"].astype("Int64")

#replace nan values with empty lists
funders_df["causes"] = funders_df["causes"].apply(lambda x: x if isinstance(x, list) else [])
funders_df["areas"] = funders_df["areas"].apply(lambda x: x if isinstance(x, list) else [])
funders_df["beneficiaries"] = funders_df["beneficiaries"].apply(lambda x: x if isinstance(x, list) else [])

#round to 2 decimal places
funders_df = funders_df.round(2)
pd.set_option("display.float_format", "{:.2f}".format)

#format financial columns
float_cols = ["income", "expenditure", "total_given", "avg_grant", "median_grant"]
for col in float_cols:
    if col in funders_df.columns:
        funders_df[col + "_gbp"] = funders_df[col].apply(add_gbp_columns)

funders_df2 = funders_df.copy()

funders_df2 = funders_df2.drop(columns=float_cols)

In [5]:
funders_df2 = funders_df2.sort_values("total_given_gbp", ascending=False)

In [6]:
funders_df2.head()

,registered_num,name,website,activities,objectives,causes,areas,beneficiaries,num_grants,income_gbp,expenditure_gbp,total_given_gbp,avg_grant_gbp,median_grant_gbp
265,200051,Esmee Fairbairn Foundation,https://www.esmeefairbairn.org.uk,We are one of the largest independent foundati...,To further such charitable purpose or purposes...,"[General Charitable Purposes, Education/traini...","[Wigan, Oxfordshire, Greater Manchester]","[Children/young People, Elderly/old People, Pe...",6967,"£9,483,000.00","£56,416,000.00","£717,939,197.42","£103,048.54","£54,000.00"
173,294629,The Fenton Arts Trust,https://www.fentonartstrust.org.uk,Awards grants to individuals and organisations...,(a) the advancement of public education more p...,[Arts/culture/heritage/science],"[Warrington, Nottingham City, Oxfordshire, Lei...",[Other Defined Groups],175,"£180,898.00","£178,450.00","£663,582.00","£3,791.90","£4,000.00"
489,274100,The Clothworkers' Foundation,https://clothworkersfoundation.org.uk,To make grants to uk frontline charities to su...,For the advancement of such charitable purpose...,[General Charitable Purposes],[],[Other Charities Or Voluntary Bodies],3237,"£12,329,000.00","£12,607,000.00","£66,349,023.60","£20,497.07","£10,000.00"
488,263207,John Ellerman Foundation,https://www.ellerman.org.uk,John ellerman foundation is a general grantmak...,1) the object of the charity is for such chari...,[General Charitable Purposes],[],[Other Charities Or Voluntary Bodies],764,"£3,431,000.00","£6,239,000.00","£65,215,922.90","£85,361.16","£90,000.00"
468,283813,The London Marathon Charitable Trust Limited,https://www.londonmarathonfoundation.org,"London marathon foundation, the operating name...",2.1 the charity's objects are restricted speci...,"[Disability, Amateur Sport, Human Rights/relig...",[],"[Children/young People, Elderly/old People, Pe...",775,"£62,715,273.00","£57,810,948.00","£62,403,416.00","£80,520.54","£30,000.00"
